In [7]:
import networkx as nx
import pandas as pd
import os
import time
import heapq

def dijkstra(graph, start):
    # Initialize distances with infinity for all nodes except the start node
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    # Initialize a dictionary to store the previous node in the shortest path
    previous_node = {node: None for node in graph}

    # Priority queue to store nodes with their current distance from start
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # Ignore outdated entries in the priority queue
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            #print(current_distance)
            #print(weight['weight'])
            distance = current_distance + weight['weight']
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_node[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    # Construct the shortest paths
    shortest_paths = {node: [] for node in graph}
    for node in graph:
        current_node = node
        while previous_node[current_node] is not None:
            shortest_paths[node].insert(0, current_node)
            current_node = previous_node[current_node]
        if current_node == start:
            shortest_paths[node].insert(0, start)

    return distances, shortest_paths

def bidirectional_dijkstra(graph, source, target, input_intersection):
    # Run Dijkstra's algorithm from both directions
    path_forward_length, path_forward = dijkstra(graph, source)
    path_backward_length, path_backward = dijkstra(graph, target)
    intersection = set(path_forward.keys()) & set(path_backward.keys())
    desired_intersection = input_intersection
    # If there is a common node, calculate the shortest path
    """print(intersection)
    print(path_forward.keys())
    print(path_backward.keys())
    print(path_forward)
    print(path_backward)
    print(path_forward_length)
    print(path_backward_length)"""
    if intersection:
        shortest_distance = float('inf')
        shortest_path = None
        shortest_path_with_desired_intersection = None
        shortest_distance_with_desired_intersection = None
        # Dijkstra's Algorithm
        for node in intersection:
            distance = path_forward_length[node] + path_backward_length[node] #adding distance in foward and backward
            if distance < shortest_distance:
                #print(str(distance)+"Distance")
                #print(str(shortest_distance)+"Sh Distance")
                if node == source or node == target:
                    continue
                elif node == desired_intersection:
                    shortest_distance = distance
                    path1 = path_forward[node][0:]
                    path2 = path_backward[node][::-1]
                    shortest_path  = path1 + [node for node in path2 if node not in path1]
                    if desired_intersection in shortest_path:
                        shortest_path_with_desired_intersection = shortest_path
                        shortest_distance_with_desired_intersection = shortest_distance
                    else:
                        continue
        #Repeating Dijkstras Algorithm for same distance
        other_distance = 0
        other_shortest_path = None
        new_shortest_path = []
        for node in intersection: 
            other_distance = path_forward_length[node] + path_backward_length[node]
            if other_distance != shortest_distance_with_desired_intersection:
                #print("1")
                continue
            else:
                if node == source or node == target:
                    continue
                elif node == desired_intersection:
                    #shortest_distance = distance
                    path1 = path_forward[node][0:]
                    path2 = path_backward[node][::-1]
                    other_shortest_path  = path1 + [node for node in path2 if node not in path1]
                    if other_shortest_path == shortest_path_with_desired_intersection:
                        #print("2")
                        continue
                    elif desired_intersection in other_shortest_path:
                        #print("3")
                        new_shortest_path.append(other_shortest_path)
        #print(other_shortest_path)
        return shortest_path_with_desired_intersection, shortest_distance, new_shortest_path
    else:
        return None, float('inf')  # No path found"""

# Example usage:
# Specify the path to your Excel file
excel_file_path = 'C:\\Users\\Admin\\Documents\\Coding Portfolio\\Python\\Codes_and_Data_Thesis_2\\Codes_and_Data_Thesis_2\\Time_Matrix.xlsx'

# Read the Excel file into a pandas DataFrame without header
df = pd.read_excel(excel_file_path, sheet_name='Sheet1', header=None)

# Remove the header and frame
df = df.iloc[0:, :].reset_index(drop=True)

# Create a graph from the DataFrame
G = nx.Graph()
for i in range(len(df)):
    for j in range(len(df.iloc[i])):
        if df.iloc[i, j] != 0:
            G.add_edge(i, j, weight=df.iloc[i, j])

while True:
    start_node = int(input("Enter your Starting Node: "))
    end_node = int(input("Enter your Destination Node: "))
    desired_intersection = int(input("Enter your Desired Intersection: "))
    start_time = time.time()
    shortest_path, shortest_distance, other_path = bidirectional_dijkstra(G, start_node, end_node, desired_intersection)
    if shortest_path:
        print(f"Shortest path from {start_node} to {end_node}: {shortest_path}")
        print(f"Other Path: {other_path}")
        print(f"Shortest distance: {shortest_distance}")
    else:
        print(f"No path exists between {start_node} and {end_node}.")
    
    end_time = time.time()
    final_time = end_time - start_time
    
    print("Elapsed time: ", final_time)
    while True:
        ifnoInput = None
        rerun = input("Do you want to run the program again? (yes/no): ")
        if rerun.lower() == 'no':
            ifnoInput = rerun
            break
        elif rerun.lower() == 'yes':
            print("\n")
            break
        else:
            continue
    if ifnoInput == 'no':
        break

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

# 